In [ ]:
import os
import datetime as dt
import glob

import gmsh
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
import numpy as np
from scipy.interpolate import RectBivariateSpline
from scipy.spatial import KDTree
from shapely.geometry import Polygon
from tqdm import tqdm


from sialib import remove_short_edges, remove_long_edges, create_mesh_from_points, get_triangulation, remove_filpped_elements

np.seterr(divide='ignore', invalid='ignore')
%matplotlib inline

In [ ]:
def make_polygon_shapely(x, y):
    return Polygon([(x[0],y[0]), (x[1],y[1]), (x[2],y[2])])

def jacobian(x0, y0, x1, y1, x2, y2):
    """
    jac = jacobian(x0, y0, x1, y1, x2, y2):
    calculates jac = det(M),
    where M is the matrix
    [[x1-x0, y1-y0], [x2-x0, y2-y0]].

    This is twice the area of a triangle with vertices:
    (x0, y0), (x1, y1), (x2, y2)

    Parameters:
    x0, x1, x2 (numpy arrays or floats) - x coords of the 3 points
    y0, y1, y2 (numpy arrays or floats) - y coords of the 3 points

    Returns:
    jac (same type as inputs)
    """
    return (x1-x0)*(y2-y0)-(x2-x0)*(y1-y0)

def get_area(x, y, t):
    return .5*jacobian(x[t][:,0], y[t][:,0], x[t][:,1], y[t][:,1], x[t][:,2], y[t][:,2])

def resample_velocities(u, v, m, xc, yc, x, y, max_m = 0.5):
    # resample velocities from grid to mesh
    u = RectBivariateSpline(xc[1:-1:3], yc[1:-1:3], u[::-1], kx=1, ky=1)(y, x, grid=False)
    v = RectBivariateSpline(xc[1:-1:3], yc[1:-1:3], v[::-1], kx=1, ky=1)(y, x, grid=False)
    m = RectBivariateSpline(xc, yc, m[::-1], kx=1, ky=1)(y, x, grid=False)
    m = np.where(m < max_m)[0]
    u[m] = 0
    v[m] = 0
    return u, v, m

def get_same_elements(t0, t1):
    set0 = set([tuple(i) for i in t0.tolist()])
    set1 = set([tuple(i) for i in t1.tolist()])
    set_inter = set0.intersection(set1)
    set0_idx = {tuple(j):i for i,j in enumerate(t0.tolist())}
    set1_idx = {tuple(j):i for i,j in enumerate(t1.tolist())}
    same_elems_in0 = np.array([set0_idx[i] for i in set_inter])
    same_elems_in1 = np.array([set1_idx[i] for i in set_inter])
    return same_elems_in0, same_elems_in1

In [ ]:
mesh_init_file = 'mesh_arctic_ease_25km_max7.npz'
xc = np.load(mesh_init_file)['xc']
yc = np.load(mesh_init_file)['yc']
mask = np.load(mesh_init_file)['mask']

In [ ]:
force = True
min_edge_length = 13
max_edge_length = 38
min_triangles = 5
search_dist = 50

sid_dir = '/Data/sim/data/OSISAF_ice_drift_CDR_postproc'
lag_dir = '/data2/antonk/sia/cdr_1991_2023/mesh'
ifiles = sorted(glob.glob(f'{sid_dir}/*/*npz'))

idates = [dt.datetime.strptime(os.path.basename(ifile).split('-')[-1].split('.')[0], '%Y%m%d%H%M%S')
          for ifile in ifiles]
print(len(ifiles), idates[0], idates[-1])

In [ ]:
# MOVE MESH
#for ifile, idate in tqdm(zip(ifiles, idates), total=len(ifiles)):
ifile = ifiles[0]
idate = idates[0]

def get_mesh_files(idate, lag_dir):
    mesh_src_dir = idate.strftime(f'{lag_dir}/%Y')
    mesh_src_file = idate.strftime(f'{mesh_src_dir}/mesh_%Y%m%d.npz')
    if not os.path.exists(mesh_src_file):
        print('Load INIT')
        mesh_src_file = mesh_init_file

    mesh_dst_date = idate + dt.timedelta(1)
    mesh_dst_dir = mesh_dst_date.strftime(f'{lag_dir}/%Y')
    os.makedirs(mesh_dst_dir, exist_ok=True)
    mesh_dst_file = mesh_dst_date.strftime(f'{mesh_dst_dir}/mesh_%Y%m%d.npz')

    return mesh_src_file, mesh_dst_file

def load_data(ifile):
    with np.load(ifile) as d:
        u = d['u']
        v = -d['v']
    u[np.isnan(u)] = 0
    v[np.isnan(v)] = 0
    with np.load(mesh_src_file) as d:
        x0 = d['x']
        y0 = d['y']
        t0 = d['t']
    return u, v, x0, y0, t0

def get_xyt1(x0, y0, t0, u, v):
    u0, v0, m0 = resample_velocities(u, v, mask, xc, yc, x0, y0)
    x1 = x0 + u0
    y1 = y0 + v0
    t1 = np.array(t0)
    return x1, y1, t1, m0

def remesh(x1, y1, t1, m0):
    x2, y2, t2, n = remove_filpped_elements(x1, y1, t1)
    print(f'Removed {n} flipped elements')

    tri2 = Triangulation(x2, y2, t2)
    tri3, n = remove_short_edges(tri2, min_edge_length, fixed_nodes_idx=m0)
    print(f'Removed {n} short edges')

    tri4, n = remove_long_edges(tri3, max_edge_length, fixed_nodes_idx=m0)
    print(f'Removed {n} long edges')
    return tri4

def optimize_mesh(tri4, same_in_4, min_triangles):
    new_elems = np.ones(tri4.triangles.shape[0], dtype=bool)
    new_elems[same_in_4] = False
    new_elems[tri4.neighbors[new_elems].flatten()] = True

    tri5 = Triangulation(tri4.x, tri4.y, tri4.triangles[new_elems])
    create_mesh_from_points(tri5)
    gmsh.model.mesh.optimize("Laplace2D", niter=1)
    tri6 = get_triangulation()

    triangles_per_node = np.zeros(tri5.x.size)
    np.add.at(triangles_per_node, tri5.triangles.flatten(), 1)

    # optimised node positions and full mesh
    x7 = np.array(tri4.x)
    y7 = np.array(tri4.y)
    t7 = np.array(tri4.triangles)
    x7[triangles_per_node >= min_triangles] = tri6.x[triangles_per_node >= min_triangles]
    y7[triangles_per_node >= min_triangles] = tri6.y[triangles_per_node >= min_triangles]
    print(f'Optimised {np.sum(triangles_per_node >= min_triangles)} nodes')
    return x7, y7, t7

def compute_mapping(x1, y1, t1, x7, y7, t7, same_in_1, same_in_4, search_dist):
    new_elems = np.ones(t7.shape[0], dtype=bool)
    new_elems[same_in_4] = False
    new_elem_idx = np.nonzero(new_elems)[0]
    src2dst = np.column_stack([same_in_1, same_in_4]).tolist()
    weights = [1] * len(src2dst)
    elem_x1 = x1[t1].mean(axis=1)
    elem_y1 = y1[t1].mean(axis=1)
    tree1 = KDTree(np.column_stack([elem_x1, elem_y1]))
    for i in new_elem_idx.tolist():
        x = x7[t7[i]].mean()
        y = y7[t7[i]].mean()
        near_elems_1 = tree1.query_ball_point([x, y], search_dist)
        p7 = make_polygon_shapely(x7[t7[i]], y7[t7[i]])
        for j in near_elems_1:
            p1 = make_polygon_shapely(x1[t1[j]], y1[t1[j]])
            p3int = p7.intersection(p1)
            if p3int.area > 0:
                src2dst.append((j, i))
                weights.append(p3int.area / p7.area)
    src2dst = np.array(src2dst)
    weights = np.array(weights)
    return src2dst, weights

def get_area_ratio(x0, y0, t0, x1, y1, t1, x7, y7, t7, src2dst, weights):
    area0 = get_area(x0, y0, t0)
    area1 = get_area(x1, y1, t1)
    area_ratio1 = area1 / area0
    area_ratio7 = np.zeros(t7.shape[0]) + 0
    np.add.at(area_ratio7, src2dst[:,1], area_ratio1[src2dst[:,0]] * weights)
    return area_ratio7

mesh_src_file, mesh_dst_file = get_mesh_files(idate, lag_dir)
#if os.path.exists(mesh_dst_file) and not force:
#    continue
u, v, x0, y0, t0 = load_data(ifile)
x1, y1, t1, m0 = get_xyt1(x0, y0, t0, u, v)
tri4 = remesh(x1, y1, t1, m0)
same_in_1, same_in_4 = get_same_elements(t1, tri4.triangles)
x7, y7, t7 = optimize_mesh(tri4, same_in_4, min_triangles)
src2dst, weights = compute_mapping(x1, y1, t1, x7, y7, t7, same_in_1, same_in_4, search_dist)
area_ratio7 = get_area_ratio(x0, y0, t0, x1, y1, t1, x7, y7, t7, src2dst, weights)

#print(f'Save mesh file to {mesh_dst_file}')
#np.savez(mesh_dst_file, x=x7, y=y7, t=t7, src2dst=src2dst, weights=weights, ar=area_ratio7)


In [ ]:
plt.tripcolor(x7, y7, t7, area_ratio7, cmap='coolwarm')

In [ ]:
tri4.triangles.shape, t7.shape